In [ ]:
import pandas as pd
import numpy as np 
import chinese_calendar
import matplotlib.pyplot as plt
import seaborn as sns
# 设置支持中文的字体，如 SimHei（黑体），Hiragino Sans GB（冬青黑体）
plt.rcParams['font.family'] = 'Hiragino Sans GB'
# 解决负号显示问题
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from chinese_calendar import is_holiday, get_holiday_detail, get_workdays

In [ ]:
#数据处理
df=pd.read_excel("国债到期收益率.xlsx",sheet_name="Sheet1",header=1)

#生成周度均值
df['指标名称'] = pd.to_datetime(df['指标名称'])
df.set_index('指标名称', inplace=True) #确定索引

#重采样生成周度均值
weekly_means = df.resample('W').mean()
weekly_df = weekly_means.reset_index()#重制索引

column_names = [1,3,5,7,10,30]

for col in column_names:
    weekly_df[f'{col}Y_diff'] = weekly_df[f'中债国债到期收益率:{col}年'].diff()*100

weekly_df["30Y_diff"].plot(kind="line").axhline(y=1,color='r',linestyle='--')

In [ ]:
#目标高精度
def assign_status_precise(value):
    if value > 1:
        return 3
    elif value <-1:
        return 2
    else:
        return 1

#目标高胜率
def assign_status_value(value):
    if value > 0:
        return 3
    else:
        return 2

In [ ]:
weekly_df_p = weekly_df.copy().dropna()
weekly_df_v = weekly_df.copy().dropna()

In [ ]:
for col in weekly_df.columns[-6:]:
    weekly_df_p[f'status_{col}'] = weekly_df[f'{col}'].apply(assign_status_precise)
    weekly_df_v[f'status_{col}'] = weekly_df[f'{col}'].apply(assign_status_value)

In [ ]:
#创建滞后值


#生成滞后值，并防止df碎片化

# 创建存储滞后列的列表
lagged_columns_30Y = []


# 生成滞后 56期的列
for i in range(1, 57):
    lagged_columns_30Y.append(weekly_df_p['status_30Y_diff'].shift(i).rename(f'status_30Y_lag_{i}'))
    
    
# 合并所有滞后列
lagged_df = pd.concat(lagged_columns_30Y,axis=1)

# 将滞后列合并到原始 DataFrame
weekly_df_p = pd.concat([weekly_df_p, lagged_df], axis=1)

weekly_df_p=weekly_df_p.dropna()

In [ ]:
# X是特征，y目标变量
X = weekly_df_p.iloc[:,-56:-36]
X['T']=range(1, len(X) + 1)
y = weekly_df_p.iloc[:, 18]  # 选取当前值作为目标变量

In [ ]:
# 数据划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 创建随机森林分类
model = RandomForestClassifier(n_estimators=20000, random_state=42,max_depth=6)
                         

# 模型训练
model.fit(X_train, y_train)

# 模型预测
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

# 模型评估-测试集
accuracy = accuracy_score(y_test, y_pred)
print(f"测试集模型准确率: {accuracy}")

# 模型评估-训练集
accuracy = accuracy_score(y_train, y_pred_train)
print(f"训练集模型准确率: {accuracy}")

# 绘制混淆矩阵
cm = confusion_matrix(y_test, y_pred)
# 获取唯一原始标签
unique_labels = sorted(set(y_test).union(set(y_pred)))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=unique_labels, yticklabels=unique_labels)
plt.xlabel('预测标签')
plt.ylabel('真实标签')
plt.title('测试集混淆矩阵')
#plt.savefig('R001-OMO测试集混淆矩阵.png', dpi=300, bbox_inches='tight')
plt.show()


# 绘制混淆矩阵
cm = confusion_matrix(y_train, y_pred_train)
# 获取唯一原始标签
unique_labels = sorted(set(y_train).union(set(y_pred_train)))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=unique_labels, yticklabels=unique_labels)
plt.xlabel('预测标签')
plt.ylabel('真实标签')
plt.title('训练集混淆矩阵')
#plt.savefig('R001-OMO训练集混淆矩阵.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
T=X.iloc[:,-1]
t=T.iloc[-1]+1
X_out=X.iloc[-1,:-2].to_frame().T
X_out.insert(0,"new_column",y.iloc[-1])
X_out.insert(20,"T",t)

if len(X_out.columns) == len(X.columns):
# 替换列名
    X_out.columns = X.columns
else:
    print("df1 和 df2 的列数不同，无法直接替换列名。")

# 用于存储每次的预测结果。注意不能重复运行，X_out的数值会改变。
predictions = []

n=15
# 迭代预测n期
for _ in range(n):
    # 进行预测
    next_prediction = model.predict(X_out)
    #把预测值放进集合
    predictions.append(next_prediction[0])
    X_out=X_out.iloc[-1,:-2].to_frame().T
    t+=1
    X_out.insert(0,"new_column",next_prediction[0])
    X_out.insert(20,"T",t)
    X_out.columns = X.columns
    
print("迭代n期的预测结果:", predictions,"注意：预测效力会随着外推减弱")